In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

import os

llm = ChatOpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_SK'), model='deepseek-chat')
print('大模型初始化成功')

# 预定
def booking_handler(request: str) -> str:
    """模拟预定智能体处理器"""
    print('委托给预定处理器')
    return f"预定处理器已接受请求: {request}, 结果：模拟预定动作"


# 信息检索
def info_handler(request: str) -> str:
    """模拟信息检索智能体处理器"""
    print('委托给信息检索处理器')
    return f"信息检索处理器已接受请求: {request}, 结果：模拟信息检索"

# 兜底
def unclear_handler(request: str) -> str:
    """处理无法委托的智能体处理器"""
    print('处理不明确的请求')
    return f"协调者无法完成委托: {request}, 结果：请补充说明"

# 定义协调者路径
cooperator_prompt = ChatPromptTemplate.from_messages([
    ("system", """
        分析用户请求，判断由哪个执行器进行处理.
        - 如果涉及预定机票或酒店，输出'booker'.
        - 如果涉及一般信息问题，输出'info'.
        - 如果需求不明确或者不属于上述类型, 输出'unclear'.
        只输出一个词: 'booker'、'info'或'unclear'.
    """),
    ("user", "{request}")
])

cooperator_chain = cooperator_prompt | llm | StrOutputParser()

branches = {
    'booker': RunnablePassthrough.assign(output=lambda x:booking_handler(x['request']['request'])),
    'info': RunnablePassthrough.assign(output=lambda x:info_handler(x['request']['request'])),
    'unclear': RunnablePassthrough.assign(output=lambda x:unclear_handler(x['request']['request'])),
}

delegation_branch = RunnableBranch(
    (lambda x: x['decision'].strip() == 'booker', branches['booker']),
    (lambda x: x['decision'].strip() == 'info', branches['info']),
    branches['unclear'],
)

cooperator_agent = {
    'decision': cooperator_chain,
    'request': RunnablePassthrough()
} | delegation_branch | (lambda x: x['output'])


def main():
    request_a = '帮我预定7点去北京的机票'
    result_a = cooperator_agent.invoke({'request': request_a})
    print(f'问题: {request_a}, 结果: {result_a}')

    request_b = '意大利的首都是哪里'
    result_b = cooperator_agent.invoke({'request': request_b})
    print(f'问题: {request_b}, 结果: {result_b}')

    request_c = '讲讲量子物理'
    result_c = cooperator_agent.invoke({'request': request_c})
    print(f'问题: {request_c}, 结果: {result_c}')


if __name__ == '__main__':
    main()

大模型初始化成功
委托给预定处理器
问题: 帮我预定7点去北京的机票, 结果: 预定处理器已接受请求: 帮我预定7点去北京的机票, 结果：模拟预定动作
委托给信息检索处理器
问题: 意大利的首都是哪里, 结果: 信息检索处理器已接受请求: 意大利的首都是哪里, 结果：模拟信息检索
委托给信息检索处理器
问题: 讲讲量子物理, 结果: 信息检索处理器已接受请求: 讲讲量子物理, 结果：模拟信息检索
